In [71]:
# 
is_on_colab = True

is_show_colab_env_info = False


if(is_on_colab):

    if(is_show_colab_env_info):

        
        def printm():

            process = psutil.Process(os.getpid())
            print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
            print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

        # memory footprint support libraries/code
        !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
        !pip install gputil
        !pip install psutil
        !pip install humanize  

        import psutil
        import humanize
        import os
        import GPUtil as GPU
        GPUs = GPU.getGPUs()
        # XXX: only one GPU on Colab and isn’t guaranteed
        gpu = GPUs[0]


        #GPU count and name
        !lscpu |grep 'Model name'
        !nvidia-smi -L
        printm()

  
    from google.colab import drive
    drive.mount('/content/drive')

    gdrive_home_dir = '/content/drive/My Drive/'

    project_dir = f'{gdrive_home_dir}DeepCNC/'
    ds_dir = f'{project_dir}data/'
    snapshot_dir = f'{project_dir}snapshots/'

    !ls '{ds_dir}'

else:
    project_dir = f'/disks/data/paper_projects/DeepCNC/'
    ds_dir = f'{project_dir}data/'
    snapshot_dir = f'{project_dir}snapshots/'

  

In [73]:
im_savedir = f'{ds_dir}imdata/'
ds_split_dir = f'{ds_dir}imdata_split_90_10/'

import glob
class_dirs = glob.glob(im_savedir+"*")
class_labels = [d[d.rfind('/')+1:] for d in class_dirs]
print(class_labels)

# make the directory for storing this split
import os
os.makedirs(ds_split_dir, exist_ok = True)

splitlabels = ['train','val']
splitratios = [0.9, 0.1]
assert sum(splitratios)==1

import random
import shutil
import math

for i in range(len(class_dirs)):
    classdir = class_dirs[i]
    class_label = class_labels[i]
    images = glob.glob(classdir+"/*.bmp")
    print(classdir,":",len(images))
    im_count = len(images)
    class_data_indices = [i for i in range(im_count)]
    shuffled_indices = class_data_indices
    random.shuffle(shuffled_indices)
    
    last_idx = 0
    for j in range(len(splitlabels)):
        splitlabel = splitlabels[j]
        splitratio = splitratios[j]
        
        splitdir = f'{ds_split_dir}{splitlabel}/'
        split_label_dir = f'{splitdir}{class_label}/'
        os.makedirs(split_label_dir, exist_ok = True)
        
        im_count_in_split_class = math.ceil(im_count*splitratio)
        split_im_indices = shuffled_indices[last_idx:last_idx + im_count_in_split_class]
        last_idx += im_count_in_split_class
        
        split_label_images = [ images[i] for i in split_im_indices ]
        #_ = [shutil.copy2(f,split_label_dir) for f in split_label_images]
        _ = [shutil.copy2(f,split_label_dir) for f in split_label_images]

        print(splitlabel,class_label,len(split_im_indices))
    

['1', '0']
/disks/data/paper_projects/DeepCNC/data/imdata/1 : 18487
train 1 16639
val 1 1848
/disks/data/paper_projects/DeepCNC/data/imdata/0 : 18787
train 0 16909
val 0 1878


In [74]:
# verify the split

indices = []
for i in range(len(class_dirs)):
    classdir = class_dirs[i]
    class_label = class_labels[i]
    class_images = glob.glob(classdir+"/*.bmp")
    class_images_count = len(class_images)
    print(class_label,"(",class_images_count,")")
        
    class_indices = []
    for j in range(len(splitlabels)):
        splitlabel = splitlabels[j]
        splitratio = splitratios[j]
        
        splitdir = f'{ds_split_dir}{splitlabel}/'
        split_label_dir = f'{splitdir}{class_label}/'
        
        split_label_images = glob.glob(split_label_dir+"/*.bmp")
        class_indices.append(set(split_label_images))
        split_label_image_count = len(split_label_images)
        print(splitlabel,"",class_label,"",split_label_image_count/class_images_count, "(",split_label_image_count,")")
        
    indices.append(class_indices)    

1 ( 18487 )
train  1  0.9000378644452859 ( 16639 )
val  1  0.09996213555471413 ( 1848 )
0 ( 18787 )
train  0  0.9000372598073135 ( 16909 )
val  0  0.09996274019268643 ( 1878 )


In [68]:
indices[1][0].intersection(indices[1][1])

set()